## Import Libraries

In [1]:
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint

## Loading Gender Predictor File to predict Gender of LinkedIn Profiles 

In [ ]:
# %load Gender_Prediction.py

# Import Libraries

import os
import re
import urllib2
from zipfile import ZipFile
import csv
import cPickle as pickle

def downloadNames():
    u = urllib2.urlopen('https://www.ssa.gov/oact/babynames/names.zip')
    localFile = open('names.zip', 'w')
    localFile.write(u.read())
    localFile.close()

def getNameList():
    if not os.path.exists('names.pickle'):
        print 'names.pickle does not exist, generating'
        
        # https://www.ssa.gov/oact/babynames/names.zip
        
        if not os.path.exists('names.zip'):
            print 'names.zip does not exist, downloading from github'
            downloadNames()
        else:
            print 'names.zip exists, not downloading'
        
        print 'Extracting names from names.zip'  
        
        namesDict=extractNamesDict()
        
        maleNames=list()
        femaleNames=list()
        
        print 'Sorting Names'
        
        for name in namesDict:
            counts=namesDict[name]
            tuple=(name,counts[0],counts[1])
            if counts[0]>counts[1]:
                maleNames.append(tuple)
            elif counts[1]>counts[0]:
                femaleNames.append(tuple)
        
        names=(maleNames,femaleNames)
        
        print 'Saving names.pickle'
        fw=open('names.pickle','wb')
        pickle.dump(names,fw,-1)
        fw.close()
        print 'Saved names.pickle'
    else:
        print 'names.pickle exists, loading data'
        f=open('names.pickle','rb')
        names=pickle.load(f)
        print 'names.pickle loaded'
        
    print '%d male names loaded, %d female names loaded'%(len(names[0]),len(names[1]))
    
    return names[0],names[1]

def extractNamesDict():
    zf=ZipFile('names.zip', 'r')
    filenames=zf.namelist()
    
    names=dict()
    genderMap={'M':0,'F':1}
    
    for filename in filenames:
        fp = zf.open(filename,'rU')
        rows=csv.reader(fp, delimiter=',', dialect=csv.excel_tab)
        try:
            for row in rows:
            #print name,row[1]
                try:
                    name=row[0].upper()
                    gender=genderMap[row[1]]
                    count=int(row[2])

                    if not names.has_key(name):
                        names[name]=[0,0]

                    names[name][gender]=names[name][gender]+count
                except:
                    pass
        except:
            pass
            
        fp.close()
        
        print '\tImported %s'%filename
    return names


def find_gender_from_first_name(name):
    if name.upper() in new_male_list:
        return "Male"
    elif name.upper() in new_female_list:
        return "Female"
    else:
        return "Unknown"    

if __name__ == '__main__':
    
	male_names, female_names = getNameList()
	new_male_list = []
	new_female_list = []

	for index,name in enumerate(male_names):
		try:
		    if (name[1]/name[2])>=4:
		        new_male_list.append(name[0])
		except:
		    new_male_list.append(name[0])
		    
	#print "Total number of Male Names after is %d." %len(new_male_list)	
	
	
	for index,name in enumerate(female_names):
		try:
		    if (name[2]/name[1])>=4:
		        new_female_list.append(name[0])
		except:
		    new_female_list.append(name[0])
		    
	#print "Total number of Female Names after is %d." %len(new_female_list)
	
	#find_gender_from_first_name('Harsh')
	


## Get Link of Profile Picture

In [3]:
def getProfilePicLink(html):
       
    soup=BeautifulSoup(html,"lxml") 
    images = [x for x in soup.find_all('img')]
    #print images
    try:
        if "shrinknp_200_200" in str(images[0]):
            imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
        else:
            imageUrlString = ""
    except:
        imageUrlString = ""
    
    #print imageUrlString
    
    return imageUrlString

## Store Profile Picture to Local Directory

In [4]:
def storeProfilePicture(profileUrl,profile_link):
    
    lst = profileUrl.split()
    userId = profile_link.split('/')[-1]
    regex=re.compile(r'(src).*')
    img_url = re.sub('src=','', "".join([m.group(0) for l in lst for m in [regex.search(l)] if m]))
    img_url = img_url.strip('""')
    #print img_url
    if img_url:
        urllib.urlretrieve(img_url, "Images/" + userId + ".jpg")
        print userId + ".jpg is saved."
        return img_url
    else:
        with open('ghost_person.png', 'rb') as f:
            data = f.read()

        with open("Images/" + userId + ".png", 'wb') as f:
            f.write(data)

        print userId + ".png is saved."
        return "https://static.licdn.com/scds/common/u/images/themes/katy/ghosts/person/ghost_person_100x100_v1.png".strip("")

## Scrap the UserID from Recommended Users' list. 

In [5]:
def getRecommendedUserIds(html):
    
    
    soup=BeautifulSoup(html,"lxml")
    #content = driver.page_source
    #images = [x for x in soup.find_all('img')]
    profLinks = [x for x in soup.find_all('li',{'class': 'profile-card'})]
    
    recUserIds = []
    #print profLinks
    for link in profLinks:
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(link))
        recId = urls[0].split('?')[0].split('/')[-1]
        recUserIds.append(recId)
    
    return recUserIds

## Get Full Name from title in source

In [6]:
def getName(html):
    
    soup=BeautifulSoup(html,"lxml")
    title = soup.find('title')
    name = str(title).replace('<title>','')
    full_name = name.replace(' | LinkedIn</title>','')
    
    return full_name

## Get All Bachelor Degree List and Make a dictionary

In [8]:
def getBachelorList():

    BachelorDict = {}
    regex = re.compile('[^a-zA-Z/]')
    with open('bachelor_degrees.txt') as fp:
        for line in fp.readlines():
            lineSepator = line.split('(')
            abbr = regex.sub('', lineSepator[1])
            abbr = abbr.split('/')
            for abrv in abbr:
                BachelorDict[abrv] = lineSepator[0].strip()
        fp.close()

    return BachelorDict

## Calculate Age from Bachelor Degree Starting Year

In [17]:
def calculate_age(bachelor_year):
    today = date.today()
    return today.year - bachelor_year + 18

## Find Person's All Degrees and their Duration 

In [10]:
def get_Degree_Duration(html):
    
    soup=BeautifulSoup(html.encode("ascii","ignore"),"lxml")
    schoolLinks = soup.find_all('li',{'class':'school'})
    degreeList = []
    time_range_list = []
    
    for soup1 in schoolLinks:

        degreeLink = soup1.find('span',{'data-field-name':"Education.DegreeName"})
        timeRange = soup1.find('span',{'class':"date-range"})

        tempDegree = str(degreeLink).replace('<span class="translated translation" data-field-name="Education.DegreeName">','')
        degree = tempDegree.replace('</span>','')

        tempTime = str(timeRange).replace('<span class="date-range">','')
        time = tempTime.replace('<time>','')
        temp_time = time.replace('</time>','')
        time_range = temp_time.replace('</span>','')

        degreeList.append(degree)
        time_range_list.append(time_range)
    
    return degreeList,time_range_list

## Find the Bachelor Year from Degree List and its Duration

In [65]:
def find_bachelor_year(degree_list,time_list):
    
    bachelor_degree_duration = set()
    
    BachelorDict = getBachelorList()
    
    for index,dg in enumerate(degree_list):

        for key in BachelorDict.keys():
            if key in dg:
                bachelor_degree_duration.add(time_list[index])
                break

        for value in BachelorDict.values():
            if value in dg:
                bachelor_degree_duration.add(time_list[index])
                break
        
        
    #print time_list   
    bachelor_degree_duration = list(bachelor_degree_duration)
    
    #print bachelor_degree_duration[0]
    
    try:
        if bachelor_degree_duration[0] == 'None':
            if time_list:
                #print time_list
                bachelor_year = int(time_list[0].split()[0]) - 5
            else:
                bachelor_year = None

        else:
            bachelor_year = int(bachelor_degree_duration[0].split()[0])    
    
    except:
        bachelor_year = None
    

    return bachelor_year

## Find the age from LinkedIn Profile

In [66]:
def age_from_linkedin_profile(profileUrl):
    
    try:
        degree_list, time_list = get_Degree_Duration(profileUrl)
    
        refined_degree_list = []
        regex = re.compile('[^a-zA-Z\s+]')

        for degree in degree_list:
            refined_degree_list.append(regex.sub('',degree))

        bachelor_year = find_bachelor_year(refined_degree_list,time_list)
        #print bachelor_year

        if bachelor_year:
            age = calculate_age(bachelor_year)
        else:
            age = None
    
    except:
        age = None
        
    return age

## Main Method

In [70]:
if __name__ == '__main__':
    
    directory = "Images"
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    userIds = ["harshparikh1001","george-kramp-8422581","marissamayer","xulifeng","ramyamadhihallysreenidhi"]
    
    recommended_profile_ids = []
    
    profileDict = {}
    #profiles = []
    
    output = open("linkedin_profiles.pickle", 'wb')   # Save all profiles as pickle file
    
    for index,usrid in enumerate(userIds):
        
        profileUrl = "https://www.linkedin.com/in/" + usrid
        
        driver = webdriver.PhantomJS()
        driver.get(profileUrl)
        html=driver.page_source
        
        profileDict[usrid] = {}
        profileDict[usrid]['Full_Name'] = getName(html)
        profileDict[usrid]['Gender'] = find_gender_from_first_name(getName(html).split()[0])
        profileDict[usrid]['Recommended_Ids'] = getRecommendedUserIds(html)
        
        profilePicUrl = getProfilePicLink(html)
        
        picUrl = storeProfilePicture(profilePicUrl,profileUrl)
        
        profileDict[usrid]['Profile_Url'] = picUrl
        profileDict[usrid]['age'] = age_from_linkedin_profile(html)
        
        #time.sleep(5) # delays for 5 seconds
    
    pickle.dump(profileDict, output,-1)
    output.close()

harshparikh1001.jpg is saved.
george-kramp-8422581.png is saved.
marissamayer.jpg is saved.
xulifeng.jpg is saved.
ramyamadhihallysreenidhi.jpg is saved.


## Open pickle file and read stuff from it

In [71]:
pkl_file = open("linkedin_profiles.pickle","rb")
mydict=pickle.load(pkl_file) # errors out here
pkl_file.close()

In [72]:
mydict

{'george-kramp-8422581': {'Full_Name': 'George Kramp',
  'Gender': 'Male',
  'Profile_Url': 'https://static.licdn.com/scds/common/u/images/themes/katy/ghosts/person/ghost_person_100x100_v1.png',
  'Recommended_Ids': ['anil-issac-0151775',
   'romhild-hoogeveen-049b3b18',
   'kris-ropella-312502',
   'andrew-tochterman-5b79343',
   'srirama-venkataraman-560a0a3',
   'howard-li-284709',
   'cheryl-fay-lauria-abb83753',
   'uma-subramanian-57a3452',
   'prasad-rv-1b430a14',
   'richardkemkers'],
  'age': 43},
 'harshparikh1001': {'Full_Name': 'Harsh Parikh',
  'Gender': 'Male',
  'Profile_Url': 'https://media.licdn.com/mpr/mpr/shrinknp_400_400/AAEAAQAAAAAAAAmGAAAAJDNhZmE5ODljLTZjMDItNGUyNC1iYjYyLWMxYTJiNzE1NmYzNA.jpg',
  'Recommended_Ids': ['harshparikh',
   'parikhharsh',
   'harsh-parikh-83976879',
   'parikhharsh86',
   'harshparikhhp',
   'singhlatika',
   'vivek-basavarajappa-81a270b1',
   'dktrivedi265',
   'shouviksdutta',
   'sanketwagh',
   'parthshah7',
   'kumar-shrestha-a3955b

## Test profile of "Barbara Corcoran"

https://www.linkedin.com/in/barbaracorcoran

In [43]:
driver = webdriver.PhantomJS()
driver.get("https://www.linkedin.com/in/barbaracorcoran")
html=driver.page_source  
print html
soup=BeautifulSoup(html,"lxml") 
images = [x for x in soup.find_all('img')]
#print images
try:
    if "shrinknp_200_200" in str(images[0]):
        imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
    else:
        imageUrlString = ""
except:
    imageUrlString = ""

<!DOCTYPE html><!--[if lt IE 7]> <html lang="en" class="ie ie6 lte9 lte8 lte7 os-linux"> <![endif]--><!--[if IE 7]> <html lang="en" class="ie ie7 lte9 lte8 lte7 os-linux"> <![endif]--><!--[if IE 8]> <html lang="en" class="ie ie8 lte9 lte8 os-linux"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie ie9 lte9 os-linux"> <![endif]--><!--[if gt IE 9]> <html lang="en" class="os-linux"> <![endif]--><!--[if !IE]><!--><html lang="en" class="os-linux"><!--<![endif]--><head>
<meta charset="UTF-8">

    

<title>Barbara Corcoran | LinkedIn</title>



  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


 
 
 

 
 




 <meta name="referrer" content="origin"> 

<meta http-equiv="X-UA-Compatible" content="IE=edge">


 <meta name="pageImpressionID" content="2ddb8dd6-ec0e-47b7-a024-cc0913584230">
 
 <meta name="appName" content="chrome">
 
 
 
 
 
 
 <meta name="pageKey" content="public_profile_v3_desktop">
 


 <meta name="treeID" content="FUm9Ta4sdRRAGaHeHisAAA==">
 

 
 
 
 

 <meta name="globalTrackingUrl"

In [37]:
imageUrlString

'<img alt="Barbara Corcoran" class="image photo lazy-loaded" height="200" lazyload="true" src="https://media.licdn.com/mpr/mpr/shrinknp_400_400/p/2/000/287/37e/3945e17.jpg" width="200"/>'

In [21]:
find_gender_from_first_name(getName(html).split()[0])

'Male'

In [67]:
print age_from_linkedin_profile(html)

None


## My Profile and Age

https://www.linkedin.com/in/harshparikh1001

In [62]:
driver = webdriver.PhantomJS()
driver.get("https://www.linkedin.com/in/harshparikh1001")
html1=driver.page_source  
print html1
soup=BeautifulSoup(html1,"lxml") 
images = [x for x in soup.find_all('img')]
#print images
try:
    if "shrinknp_200_200" in str(images[0]):
        imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
    else:
        imageUrlString = ""
except:
    imageUrlString = ""

<!DOCTYPE html><!--[if lt IE 7]> <html lang="en" class="ie ie6 lte9 lte8 lte7 os-linux"> <![endif]--><!--[if IE 7]> <html lang="en" class="ie ie7 lte9 lte8 lte7 os-linux"> <![endif]--><!--[if IE 8]> <html lang="en" class="ie ie8 lte9 lte8 os-linux"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie ie9 lte9 os-linux"> <![endif]--><!--[if gt IE 9]> <html lang="en" class="os-linux"> <![endif]--><!--[if !IE]><!--><html lang="en" class="os-linux"><!--<![endif]--><head>
<meta charset="UTF-8">

    

<title>Harsh Parikh | LinkedIn</title>



  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


 
 
 

 
 




 <meta name="referrer" content="origin"> 

<meta http-equiv="X-UA-Compatible" content="IE=edge">


 <meta name="pageImpressionID" content="4e661fa9-5672-49d1-a11a-3b80419b7fd7">
 
 <meta name="appName" content="chrome">
 
 
 
 
 
 
 <meta name="pageKey" content="public_profile_v3_desktop">
 


 <meta name="treeID" content="FL5SsWstdRRAA9/THisAAA==">
 

 
 
 
 

 <meta name="globalTrackingUrl" con

In [68]:
age_from_linkedin_profile(html1)

24

## Test profile of "Satya Nadella"

https://www.linkedin.com/in/satya-nadella-3145136

In [49]:
driver = webdriver.PhantomJS()
driver.get("https://www.linkedin.com/in/satya-nadella-3145136")
html2=driver.page_source  
print html2
soup=BeautifulSoup(html2,"lxml") 
images = [x for x in soup.find_all('img')]
#print images
try:
    if "shrinknp_200_200" in str(images[0]):
        imageUrlString = str(images[0]).replace("shrinknp_200_200", "shrinknp_400_400")
    else:
        imageUrlString = ""
except:
    imageUrlString = ""

<!DOCTYPE html><!--[if lt IE 7]> <html lang="en" class="ie ie6 lte9 lte8 lte7 os-linux"> <![endif]--><!--[if IE 7]> <html lang="en" class="ie ie7 lte9 lte8 lte7 os-linux"> <![endif]--><!--[if IE 8]> <html lang="en" class="ie ie8 lte9 lte8 os-linux"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie ie9 lte9 os-linux"> <![endif]--><!--[if gt IE 9]> <html lang="en" class="os-linux"> <![endif]--><!--[if !IE]><!--><html lang="en" class="os-linux"><!--<![endif]--><head>
<meta charset="UTF-8">

    

<title>Satya Nadella | LinkedIn</title>



  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


 
 
 

 
 




 <meta name="referrer" content="origin"> 

<meta http-equiv="X-UA-Compatible" content="IE=edge">


 <meta name="pageImpressionID" content="7fd585d9-47f7-4a23-9c6c-9d59231ed336">
 
 <meta name="appName" content="chrome">
 
 
 
 
 
 
 <meta name="pageKey" content="public_profile_v3_desktop">
 


 <meta name="treeID" content="X1TkDhwtdRSAiukFHysAAA==">
 

 
 
 
 

 <meta name="globalTrackingUrl" co

In [69]:
age_from_linkedin_profile(html2)

45

In [ ]:


#driver = webdriver.PhantomJS()
#driver.get("https://www.linkedin.com/in/georgecorliss")
#html=driver.page_source    
#soup=BeautifulSoup(html,"lxml")
#schoolLinks = soup.find_all('li',{'class':'school'})

#for soup1 in schoolLinks:
    
#    degreeLink = soup1.find('span',{'data-field-name':"Education.DegreeName"})
#    timeRange = soup1.find('span',{'class':"date-range"})
    
#    tempDegree = str(degreeLink).replace('<span class="translated translation" data-field-name="Education.DegreeName">','')
#    degree = tempDegree.replace('</span>','')
    
#    tempTime = str(timeRange).replace('<span class="date-range">','')
#    time = tempTime.replace('<time>','')
#    temp_time = time.replace('</time>','')
#    time_range = temp_time.replace('</span>','')
    
#    print degree
#    print time_range

In [ ]:
#import re
#link = re.sub(r'(<a href="http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+">)','',str(links[3]))
#link = link.replace('<ul>',' ')
#link = link.replace('</li><li>',' ')
#link = link.replace('<li>',' ')
#link = link.replace('</a>',' ')
#link = link.replace('</li></ul>',' ')
#print link.split()

## Link to get all profiles with the first Name

https://www.linkedin.com/pub/dir/Bill